In [1]:
import random
bits = 128

from collections import defaultdict
from collections import namedtuple

import secrets
from typing import List, Tuple

import numpy as np

## 1

### References:
- [The ElGamal Cryptosystem](https://en.wikipedia.org/wiki/ElGamal_encryption)


In [167]:
def e_keygen(q):
    # Public key: (q, g, h)
    # (G is implicit here)
    # Private key: x
    x = random.randint(1, q - 1)
    while True:
        g = random.randint(2, q-1)
        if np.gcd(g, q) == 1:
            break
    
    h = pow(g,x,q)
    sk = x
    pk = (q, g, h)
    return sk, pk

    ##raise NotImplementedError()

In [168]:
def e_encrypt(msg, pk):
    q, g, h = pk
    m = msg
    y = random.randint(1, q - 1)
    s = pow(h,y,q)
    c1 = pow(g,y,q)
    c2 = (m * s) % q
    return (c1,c2)

    #raise NotImplementedError()

In [169]:
def e_decrypt(c, x, pk):
    c1, c2 = c
    q, g, h = pk
    s = pow(c1,x, q)
    s_inv = pow(s, -1, q)
    m = c2 * s_inv
    return m % q
    

    # YOUR CODE HERE
    #raise NotImplementedError()

In [170]:
# TEST CASE
q = 2**127 - 1

sk, pk = e_keygen(q)
c = e_encrypt(50, pk)

e_decrypt(c, sk, pk)

50

In [171]:
def e_mult(c1, c2, pk):
    q, g, h = pk
    c11, c12 = c1
    c21, c22 = c2
    c1 = (c11 * c21) % q
    c2 = (c12 * c22) % q
    return (c1, c2)
    
    # YOUR CODE HERE
    #raise NotImplementedError()

In [172]:
# TEST CASE
sk, pk = e_keygen(q)
c1 = e_encrypt(5, pk)
c2 = e_encrypt(10, pk)
c3 = e_mult(c1, c2, pk)

assert e_decrypt(c3, sk, pk) == 50

## 2

Supported operations are addition of two ciphertext numbers (encrpyted numbers) and multiplication between a ciphertext number with a plaintext number. The Data Typ has to be integer since we use modulo to encrypt.

In [173]:
PrivateKey = namedtuple("PrivateKey", ["lam", "mu"])
PublicKey = namedtuple("PublicKey", ["g", "n", "n_squared"])

DEFAULT_BIT_LENGTH = 32


def generate_primes(n: int) -> List[int]:
    # https://stackoverflow.com/questions/2068372/fastest-way-to-list-all-primes-below-n-in-python/3035188#3035188
    """ Input n>=6, Returns an array of primes, 2 <= p < n """
    sieve = np.ones(n // 3 + (n % 6 == 2), dtype=bool)
    for i in range(1, int(n ** 0.5) // 3 + 1):
        if sieve[i]:
            k = 3 * i + 1 | 1
            sieve[k * k // 3 :: 2 * k] = False
            sieve[k * (k - 2 * (i & 1) + 4) // 3 :: 2 * k] = False
    primes = np.r_[2, 3, ((3 * np.nonzero(sieve)[0][1:] + 1) | 1)]
    return [int(n) for n in primes]


def L(n: int, x: int) -> int:
    return (x - 1) // n


def create_key_pair(
    bit_length: int = DEFAULT_BIT_LENGTH,
) -> Tuple[PrivateKey, PublicKey]:
    primes = generate_primes(2 ** (bit_length // 2))

    p = secrets.choice(primes)
    q = secrets.choice(primes)
    n = p * q

    while p == q or n.bit_length() != bit_length or np.gcd(n, (p - 1) * (q - 1)) != 1:
        p = secrets.choice(primes)
        q = secrets.choice(primes)
        n = p * q

    n_squared = n ** 2
    g = secrets.randbelow(n_squared - 1) + 1
    public_key = PublicKey(g, n, n_squared)

    lam = int(np.lcm(p - 1, q - 1))

    try:
        mu = pow(L(n, pow(g, lam, n_squared)), -1, n)
    except ValueError:
        return create_key_pair(bit_length)

    private_key = PrivateKey(lam, mu)
    return private_key, public_key


def encrypt(public_key: PublicKey, plaintext: int) -> int:
    g, n, n_squared = public_key
    r = secrets.randbelow(n)
    return (pow(g, plaintext, n_squared) * pow(r, n, n_squared)) % n_squared


def decrypt(private_key: PrivateKey, public_key: PublicKey, ciphertext: int) -> int:
    lam, mu = private_key
    _, n, n_squared = public_key
    return (L(n, pow(ciphertext, lam, n_squared)) * mu) % n


def add(public_key: PublicKey, ciphertext_a: int, ciphertext_b: int) -> int:
    return (ciphertext_a * ciphertext_b) % public_key.n_squared


def multiply(public_key: PublicKey, ciphertext_a: int, plaintext_b: int) -> int:
    if plaintext_b == 0:
        return encrypt(public_key, 0)

    if plaintext_b == 1:
        encrypted_zero = encrypt(public_key, 0)
        return add(public_key, ciphertext_a, encrypted_zero)

    return pow(ciphertext_a, plaintext_b, public_key.n_squared)

In [174]:
def encrypted_price_calculator(public_key: PublicKey, encrypted_cart: List[Tuple[int, int]],) -> int:
    encrypted_total = encrypt(public_key, 0)
    for price, quantity in encrypted_cart:
        en_subtotal = multiply(public_key, price, quantity)
        encrypted_total = add(public_key, en_subtotal, encrypted_total)
    return encrypted_total


In [175]:
def shopping_cart_client(cart):
    # TODO: your code here
    sk, pk = create_key_pair()
    encrypted_cart = []
    for price, quantity in cart:
        encrypted_price = encrypt(pk, price)
        encrypted_cart.append((encrypted_price, quantity))
    
    total_en_cost = encrypted_price_calculator(pk, encrypted_cart)
    total_cost = decrypt(sk, pk, total_en_cost)
    return total_cost


In [176]:
# TEST CASE
cart = [
    # (price, quantity)
    (2000, 1),
    (120, 5),
    (1999, 3),
]

total_cost = shopping_cart_client(cart)
expected_price = 8597
assert total_cost == expected_price

## 3

In [255]:
def e_encrypted_price_calculator(en_price, en_quantity, pk):
    en_total = e_mult(en_price, en_quantity, pk)
    return en_total


In [256]:
def e_shopping_cart_client(cart):
    q = 2**127 - 1
    sk, pk = e_keygen(q)
    total_en = 0
    for price, quantity in cart:
        en_price = e_encrypt(price, pk)
        en_quantity = e_encrypt(quantity, pk)
        total_en += e_decrypt(e_encrypted_price_calculator(en_price, en_quantity, pk),sk ,pk)
        
    return total_en
    

In [257]:
# TEST CASE
cart = [
    # (price, quantity)
    (2000, 1),
    (120, 5),
    (1999, 3),
]

total_cost = e_shopping_cart_client(cart)
expected_price = 8597
assert total_cost == expected_price

Changes: For ElGamal the code had to be changed so that the additions do not happen in the encypted calculator. Instead, the quantity is encrypted and multiplied by the encrypted price.

With Paillier, the calculator sees the quantity in plain text. The rest is encrypted.  For elgamal, everything is encrypted for the calculator. Therefore we have more privacy for the client with the elGamal calculator but there the client has to add the sums itself.

## 4

In [306]:
import concrete.numpy as cnp

@cnp.compiler({"x": "encrypted"})
def add(x):
    return np.sum(x) // 6

example = [[1, 7, 4, 5, 5, 4]]

circuit = add.compile(example)

result = circuit.encrypt_run_decrypt(*example)

clear_evaluation = np.mean(example)
print(f"Evaluation of mean (plain) = {clear_evaluation}, homomorphically = {result}")

Evaluation of mean (plain) = 4.333333333333333, homomorphically = 4


In [207]:
def homomorphic_mean2(x, circuit):
    encripted_array = []
    ctxtSum = 0
    for i in x:
        value = np.array([i], dtype=np.float64)
        ctxt_value = circuit.encryptFrac(value)
        encripted_array.append(ctxt_value)
        ctxtSum += ctxt_value

    mean_of_array = ctxtSum / 6
    
    resSum = circuit.decryptFrac(mean_of_array)
    
    return round(resSum[0],2)
    
def generate_cnp_circuit2():
    HE = Pyfhel()           
    ckks_params = {
        'scheme': 'CKKS',  
        'n': 2**14,                 
        'scale': 2**30,     
        'qi_sizes': [60, 30, 30, 30, 60]  
    }
    HE.contextGen(**ckks_params)  # Generate context for ckks scheme
    HE.keyGen()             # Key Generation: generates a pair of public/secret keys
    HE.rotateKeyGen()
    return HE

In [208]:
example = [1, 7, 4, 5, 5, 4]
circuit2 = generate_cnp_circuit2()
homomorphic_evaluation = homomorphic_mean2(example, circuit2)
clear_evaluation = np.mean(example)
print(f"Evaluation of mean (plain) = {clear_evaluation}, homomorphically = {homomorphic_evaluation}")

Evaluation of mean (plain) = 4.333333333333333, homomorphically = 4.33


0. With only integer we don't have floats, so we have to work with ints only, that's why we have 4 and not 4.3, it is huge limitation.
1. Also another limitation that we have to know the length of the array.
2. Moreover, to have more accurate solution we use SHE CKKS in the second implementation that allowed us have almost the same solutions, but the main difference between SHE and FHE lies in the extent of operations allowed on encrypted data without decryption. SHE allows limited operations (usually either addition or multiplication), while FHE enables a broader range of operations without compromising security.
3. We can obtain the mean using our second approach and using floats.